&nbsp;
### 调包就能解决的统计，非要手搓 ;(
### 因为没有技术含量，也没必要作注释了
### 常见的问题，多样的解决办法，选择了最笨的那些 ;(
&nbsp;
## Sunny cool humidity-high wind-strong day, but NOT the tennis day.
&nbsp;

In [8]:
import numpy as np


class Day:
    def __init__(self, outlook, temperature, humidity, wind, flight):
        self.outlook = outlook
        self.temperature = temperature
        self.humidity = humidity
        self.wind = wind
        self.flight = flight

# !! Ignore the evidence, for this time !!

class NaiveBayesBinaryClassifier:
    def statics(self, data_list, positive_label=1):
        probability_dict_list_of_negative = []
        probability_dict_list_of_positive = []
        feature_dim = 0
        feature_list = []
        label_list = []
        train_set_shape = np.array(data_list).shape
        sample_num = train_set_shape[0]
        print("训练集中的样本数:", sample_num)

        if isinstance(data_list[0], list):
            feature_dim = len(data_list[0])-1
            print("特征维度:", feature_dim)
            for data in data_list:
                feature_list.append(data[0:feature_dim])
                label_list.append(data[feature_dim])

        elif isinstance(data_list[0], object):
            feature_dim = len(data_list[0].__dict__)-1
            print("特征维度:", feature_dim)
            for data in data_list:
                feature_list.append(list(vars(data).values())[0:feature_dim])
                label_list.append(list(vars(data).values())[feature_dim])

        feature_list_of_positive = []
        feature_list_of_negative = []
        for i in range(len(feature_list)):
            if (label_list[i] == positive_label):
                feature_list_of_positive.append(feature_list[i])
            else:
                feature_list_of_negative.append(feature_list[i])

        print("正样本个数:", len(feature_list_of_positive))

        feature_set_list = []
        for j in range(len(feature_list[0])):
            feature_set = set()
            for feature in feature_list:
                feature_set.add(feature[j])
            feature_set_list.append(feature_set)

        for i in range(feature_dim):
            probability_dict_list_of_negative.append({key: 0 for key in feature_set_list[i]})
            probability_dict_list_of_positive.append({key: 0 for key in feature_set_list[i]})

        for feature in feature_list_of_negative:
            for i in range(feature_dim):
                probability_dict_list_of_negative[i][feature[i]] += 1.0

        for i in range(feature_dim):
            probability_dict_list_of_negative[i] = {key: value / len(feature_list_of_negative)
                                                    for key, value in probability_dict_list_of_negative[i].items()}

        for feature in feature_list_of_positive:
            for i in range(feature_dim):
                probability_dict_list_of_positive[i][feature[i]] += 1.0

        for i in range(feature_dim):
            probability_dict_list_of_positive[i] = {key: value / len(feature_list_of_positive)
                                                    for key, value in probability_dict_list_of_positive[i].items()}

        return (positive_label,
                feature_dim,
                len(feature_list),
                len(feature_list_of_positive),
                probability_dict_list_of_positive,
                probability_dict_list_of_negative)

    def predict(self, target, parameter_tuple):
        positive_label = parameter_tuple[0]
        feature_dim = parameter_tuple[1]
        sample_num = parameter_tuple[2]
        positive_sample_num = parameter_tuple[3]
        negative_sample_num = sample_num - positive_sample_num
        probability_dict_list_of_positive = parameter_tuple[4]
        probability_dict_list_of_negative = parameter_tuple[5]

        pi_conditional_probability = 1.0
        for i in range(feature_dim):
            pi_conditional_probability *= probability_dict_list_of_positive[i][target[i]]
        target_to_be_positive = (positive_sample_num / 1.0) / sample_num * pi_conditional_probability

        pi_conditional_probability = 1.0
        for i in range(feature_dim):
            pi_conditional_probability *= probability_dict_list_of_negative[i][target[i]]
        target_to_be_negative = (negative_sample_num / 1.0) / sample_num * pi_conditional_probability

        print("To Be Negative? ", target_to_be_negative)
        print("To Be Positive? ", target_to_be_positive)

        if (target_to_be_negative >= target_to_be_positive):
            return "Negative"
        else:
            return "Positive"

# int main()
# {

day_num = 14
day_list = []
outlook_status = ['sunny', 'overcast', 'rain']
temperature_status = ['cool', 'mild', 'hot']
humidity_status = ['high', 'normal']
wind_status = ['weak', 'strong']
fight_status = ['no', 'yes']
status_code = [[0, 2, 0, 0, 0],
               [0, 2, 0, 1, 0],
               [1, 2, 0, 0, 1],
               [2, 1, 0, 0, 1],
               [2, 0, 1, 0, 1],
               [2, 0, 1, 1, 0],
               [1, 0, 1, 1, 1],
               [0, 1, 0, 0, 0],
               [0, 0, 1, 0, 1],
               [2, 1, 1, 0, 1],
               [0, 1, 1, 1, 1],
               [1, 1, 0, 1, 1],
               [1, 2, 1, 0, 1],
               [2, 1, 0, 1, 0]]

for i in range(day_num):
    day_list.append(Day(outlook_status[status_code[i][0]],
                        temperature_status[status_code[i][1]],
                        humidity_status[status_code[i][2]],
                        wind_status[status_code[i][3]],
                        fight_status[status_code[i][4]]))

target = ['sunny', 'cool', 'high', 'strong']

print('Task 5: ')
model = NaiveBayesBinaryClassifier()
print(model.predict(target, model.statics(day_list, positive_label='yes')))
print('\n')

print('For Experiment Only: ')
parameter_tuple = model.statics([[2, 3, 0], [4, 5, 1], [1, 2, 1], [2, 3, 1]])
print(parameter_tuple)

# return 0
# }


Task 5: 
训练集中的样本数: 14
特征维度: 4
正样本个数: 9
To Be Negative?  0.02057142857142857
To Be Positive?  0.005291005291005291
Negative


For Experiment Only: 
训练集中的样本数: 4
特征维度: 2
正样本个数: 3
(1, 2, 4, 3, [{1: 0.3333333333333333, 2: 0.3333333333333333, 4: 0.3333333333333333}, {2: 0.3333333333333333, 3: 0.3333333333333333, 5: 0.3333333333333333}], [{1: 0.0, 2: 1.0, 4: 0.0}, {2: 0.0, 3: 1.0, 5: 0.0}])
